Example for Victor.

Example 1 is for an arbitrary MT at teleseicmic distance (60 degrees?).
Source depth is 15 km.

In [ ]:
# optional
# %matplotlib notebook

We search for focal mechanisms by uniformly sampling points on a unit sphere. These points are the ends of T and P axes that define a focal mechanism. 

Only a half sphere is computed; the other half will be similar/same.

In [ ]:
import numpy as np
rng = np.random.default_rng(999)  # fixed seed for reproducibility
import matplotlib.pyplot as plt
from operator import itemgetter

In [ ]:
# optional 
np.set_printoptions(precision=2, suppress=True)

In [ ]:
# This can be replaced by Caio's Ray Tracer

from obspy.taup.tau_model import TauModel
from obspy.taup.taup_create import build_taup_model
from obspy.taup import TauPyModel, plot_travel_times

In [ ]:
# This can be replaced by Caio's Ray Tracer

ak135 = TauPyModel(model='ak135')
model = ak135
model_name = 'ak135'

In [ ]:
hdepth = 15   # km   - assumed quake depth  (a parameter that affects 
# take-off angles, given a distance between quake and station)

b3_over_a3 = (3.4600/5.8000)**3    # ideally this S- over P-velocity 
# ratio cubed should be read directly from the velocity model that 
# is being used. My hope is Caio can provide this functionality. 
# The P and S velocities for this ratio are the velocities at the depth of the quake.


In [ ]:
space = [-1,1]          # for plotting "spherical" vector space
dd = np.radians(5)     # sampling step size, e.g. 2 or 5 degrees, for (grid) search
# use 5 degs for visualization, and 2 or fewer degs for searching.
n = round(16/(dd*dd))   # approximate points on sphere
n2 = round(n/2)         # points on half-sphere

In [ ]:
# What M(oment) T(ensor) to use for our fake observation 
# Global CMT convention: M_rr, M_thetatheta, M_phiphi, M_rtheta, M_rphi, M_thetaphi
mt = [2.48,0.1,-2.58,2.59,2.02,-1.12]  # arbitrary

# Definitely add code here that can visualize the beach ball corresponding to this MT

# Note to Victor: the sum of the first 3 elements (trace) is zero for a double-couple mechanism.
# do not use a MT for which you have not visualized the corresponding beach ball. 

# simpler examples:
# mt = [0,0,0,0,0,1]  # strike-slip fault
# mt = [1,-1,0,0,0,0]  # thrust fault
# mt = [-1,1,0,0,0,0]  # normal fault


# call code here from Marsquake code (on Omkar's github) to convert this MT 
# to corresponding parameters (for later reference), including
# 1. the strike, dip, rake of both planes (the fault plane and the auxiliary plane)
# 2. the azimuth and dip of each of the three axes: T, P, and N
# 3. trace of MT, CLVD* of MT, and M_0 (total moment) 
# * CLVD = Compensated linear vector dipole, ergo: a non-double couple part of the MT


In [ ]:
# fake an observation

# Use the marsquake code to calculate vector A (with P, SV and SH wave amplitudes),
# given the MT defined above, a quake depth, and
# a distance to the observing station, which should be translated to the 
# two take-off angles for P and S waves via a ray tracing package like 
# ObsPy's TauP or Caio's Ray Tracer.
# Multiply with a random number (to simulate not knowing absolute amplitudes
# and call this vector Ao ("observed")


Ao[0] = Ao[0]*b3_over_a3    # take into account the effect of seismic 
# velocities at the quake's depth
so = np.array([0.5,1.5,2.0])  # sigma -- same units (and scale) as Ao! 
# (simulated observational errors)


Aolength = np.linalg.norm(Ao)
Ao1 = Ao/Aolength   # Ao1 = Aoh (h for hat): unit vector
so1 = so/Aolength   # scale uncertainties the same as the main vector




In [ ]:
# optional
# Define error ellipsoid for later plotting

# Set of all spherical angles:
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)

# Cartesian coordinates that correspond to the spherical angles:
# (this is the equation of an ellipsoid):
xe = Ao[0] + so[0] * np.outer(np.cos(u), np.sin(v))
ye = Ao[1] + so[1] * np.outer(np.sin(u), np.sin(v))
ze = Ao[2] + so[2] * np.outer(np.ones_like(u), np.cos(v))

xe1 = Ao1[0] + so1[0] * np.outer(np.cos(u), np.sin(v))
ye1 = Ao1[1] + so1[1] * np.outer(np.sin(u), np.sin(v))
ze1 = Ao1[2] + so1[2] * np.outer(np.ones_like(u), np.cos(v))

xsax = Ao1[0] + apprxso1 * np.outer(np.cos(u), np.sin(v))
ysax = Ao1[1] + apprxso1 * np.outer(np.sin(u), np.sin(v))
zsax = Ao1[2] + apprxso1 * np.outer(np.ones_like(u), np.cos(v))

# plot in 3D with f=plt.figure(); a=f.add_subplot(projection='3d'); a.plot_surface(xe,ye,ze)

In [ ]:
# Example of how to get take-off angles from a quake depth and 
# distance from quake to to station, using ObsPy's TauP

arrivals = model.get_travel_times(source_depth_in_km=hdepth,
                         distance_in_degree=epdist, phase_list=['P', 'S'])
print(arrivals,'\n')

print("a.name, a.distance, a.time, a.ray_param, a.takeoff_angle, a.incident_angle")
for a in arrivals:
    print(a.name, a.distance, a.time, a.ray_param, a.takeoff_angle, a.incident_angle)


In [ ]:
# confirm that there is only one arrival or choose which of multiple arrivals:

Parr = arrivals[0]
Sarr = arrivals[1]

takeoff_angles = [Parr.takeoff_angle, Sarr.takeoff_angle]

In [ ]:
# old approximated error sphere radius 
# (from Sita and Van der Lee (2022) - factor 1/3 disappeared from paper)
# so1 is uncertainty scaled by the length of Ao
e_old = np.linalg.norm(so1)/np.sqrt(3)  
epsilon_old = np.arctan2(np.asarray(e_old),np.linalg.norm(Ao1))

In [ ]:
# now search through focal mechanisms by systematically rotating T and P axes:

angles = np.arange(0,np.pi,dd)  # search angles range from 0 to 180 degrees.

discarded = []
secondbest = []
accepted = []
old_accepted = []

T0 = np.array([-1,0,0])  # "southpole"
Ts = [T0]
sphcs = [np.array([0,0])]
for a in angles[1:]:    # a is "take-off angle" of T axis
                        #(measured w.r.t. downpointing vertical) ("latitude")
    ddo = dd/np.sin(a)  # need fewer sampling points on small-circle when close to the poles
    angleso = np.arange(0,np.pi,ddo)
    for o in angleso:   # o is azimuth of T axis ("longitude")
        T = np.array([-np.cos(a), -np.sin(a)*np.cos(o), np.sin(a)*np.sin(o)])
        Ts.append(T)
        sphcs.append(np.degrees(np.array([o,a])))

for T in Ts:
    P0 = mtf.perp(T)   # rotate T over 90 degrees to find one P axis
    Ps = [P0]
    for op in angles[1:]:   # find all P axes
        # rotate P0 over angle op around axis T - Rodrigues' rotation formula
        # not sure what direction this rotation is in, but it shouldn't matter
        P = P0*np.cos(op) + (np.cross(T,P0))*np.sin(op) + T*(np.dot(T,P0))*(1-np.cos(op))
        Ps.append(P)
    
    # here begins the real search work:    
    for P in Ps:
        # vectors representing T and P axes (in r-theta-phi coordinates)
        # reminder (for seismologists): r (up), theta (south), and phi (east)  i.e. Z (up) S, E system
        fault1r, fault2r = ... # get strike dip rake for two fault planes from a given T and P axis (use marquake code?)
        fault1 = [np.degrees(z) for z in fault1r] # if necessary
        fault2 = [np.degrees(z) for z in fault2r] # if necessary
        
        As = ...  # predicted AP,ASV,ASH for this T,P combo (use marsquake code)
        As[0] *= b3_over_a3  # divide by a**3 and scale with b**3  
        As1 = As/np.linalg.norm(As)

        cosos = np.dot(Ao1,As1)
        os_angle = np.arccos(cosos)     # objective function we search for minima                     
        
   
        if cosos < 1.:     # avoid cosos = 1 (perfect fit)          
            # estimate error from ellipsoid around observed amplitudes (Ao)
            v = As1 - cosos*Ao1 # this vector is the component of As that is _|_ to Ao
            vlength = np.linalg.norm(v)
            vell = v/so1        # estimate ellipsoid half width // to v
            velllen = np.linalg.norm(vell)
            # distance to ellipsoid surface along the line in the direction of v:
            e = vlength/velllen  # sigma along vector v
            epsilon = np.arctan(e) # tolerance
            # really arctan(e/||Aohat||) but the denominator = 1
            # This is an approximation of Victor's method, and better than what was
            # crudely assumed in Sita and Van der Lee (2022)
            
            if os_angle < epsilon:
                accepted.append([T,P,fault1,fault2,As,As1,
                        np.degrees(os_angle),np.degrees(epsilon),os_angle/epsilon])
            elif os_angle < 2*epsilon:
                secondbest.append([T,P,fault1,fault2,As,As1,
                        np.degrees(os_angle),np.degrees(epsilon),os_angle/epsilon])
            else:
                discarded.append([T,P,fault1,fault2,As,As1,
                        np.degrees(os_angle),np.degrees(epsilon),os_angle/epsilon])
            if os_angle < epsilon_old:  # old error sphere from paper
                old_accepted.append([T,P,fault1,fault2,As,As1,
                        np.degrees(os_angle),np.degrees(epsilon),os_angle/epsilon])
        else:   # perfect alignment
            accepted.append([T,P,fault1,fault2,As,As1,
                        np.degrees(os_angle),np.degrees(epsilon),os_angle/epsilon])
            old_accepted.append([T,P,fault1,fault2,As,As1,
                        np.degrees(os_angle),np.degrees(epsilon),os_angle/epsilon])
            print('One predicted A was perfectly aligned with the observed A')


print("DONE")


The above cell cycles through a "grid" of T,P axes combos systematically, in a way that approximates uniform coverage of a (half) sphere. The cell below provides the axes themselves, in a way that is comparable to the method in the following cell, which does it randomly. 


In [ ]:
# corresponding version of the systematic approach:systematic "grid" of axes,
# uniformly piercing a (half) sphere

# generic range for covering a half sphere
# 0 to one step before 180 degrees - 
angles = np.arange(0,np.pi,dd) 

vector0 = np.array([0,0,1])  
vectors = [vector0]
sphcs = [np.array([0,0])] 
for a in angles[1:]:    # a is co-latitude
    ddo = dd/np.sin(a)
    angleso = np.arange(0,np.pi,ddo)
    for o in angleso:   # o is longitude 
        vector = max(space)*np.array([np.sin(a)*np.sin(o), np.sin(a)*np.cos(o), -np.cos(a)])
        vectors.append(vector)
        sphcs.append(np.degrees(np.array([o,a])))

nv = len(vectors) 


In [ ]:
# random selection

randors = []
rphcs = []
rnos = rng.uniform(0,1,[n2,2])  # get random values btw 0 & 1, to be converted to co-latitude and longitude 
for r in rnos:
    a = np.arccos(1-2*r[0])     # co-latitude
    # r[0] values (theta) represent CDF* of sin(co-lat), the PDF; 
    # then write co-lat = f(y), where f is the inverse of the CDF.
    # * CDF = Cumulative probability Density Function
    o = r[1]*np.pi              # longitude
    vector = max(space)*np.array([np.sin(a)*np.sin(o), np.sin(a)*np.cos(o), -np.cos(a)])
    randors.append(vector)
    rphcs.append(np.degrees(np.array([o,a])))

nr = len(randors)


Go ahead and plot these sets of vectors (representing one side of axes)

In [ ]:
print(len(accepted))
print(len(secondbest))
print(len(discarded))
print(len(old_accepted))

In [ ]:
bins = np.arange(0,180,2)
plt.hist([a[6] for a in discarded], bins=bins, label='discarded',alpha=0.5)
plt.hist([a[6] for a in old_accepted], bins=bins, label='old accepted',alpha=0.5)
plt.hist([a[7] for a in accepted], bins=bins, label='tolerance',alpha=0.5)
plt.hist([a[6] for a in accepted], bins=bins, label='accepted',alpha=0.8)
plt.xlabel('angles')
plt.legend()

Proceed from here with beach ball visualizations, characterization, etc.